In [4]:
#!pip install youtube-search-python
#!pip install google_auth_oauthlib
#!pip install google-api-python-client

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached youtube_search_python-1.5.3-py3-none-any.whl (73 kB)
     |████████████████████████████████| 83 kB 153 kB/s             
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
     |████████████████████████████████| 59 kB 753 kB/s             
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
  Using cached h11-0.12.0-py3-none-any.whl (54 kB)
     |████████████████████████████████| 79 kB 481 kB/s             
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead

In [5]:
import pandas as pd
from os import listdir
import csv
import time

from youtubesearchpython import *

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

pd.set_option("display.max_rows", None)

In [12]:
# Get videos
filepaths = ["results/" + f for f in listdir("results/") if f.endswith(".csv")]
df_videos = pd.concat(map(pd.read_csv, filepaths))

# Get videos metadata
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("unique_videos_metadata")]
df_processed = pd.concat(map(pd.read_csv, filepaths))

In [13]:
print(df_videos.shape)
print(df_processed.shape)

(462986, 14)
(56424, 11)


In [14]:
df_processed_channel = df_processed.groupby(["channel_title", "channel_id"]).size().sort_values(ascending=False).to_frame("unique_count").reset_index()

In [15]:
df_videos=df_videos.merge(df_processed, on="video")

In [16]:
df_videos.count()

keyword                461982
video                  461982
parent_video           461802
current_depth          461982
pubdate                459424
likes                  461982
dislikes               461982
rating                 461982
reco_count             461982
fetch_time             461982
reco_list               92577
elapsed                461982
trace                  461982
parent_trace           461982
title                  461982
duration               458929
view_count             461982
width                  461982
height                 461982
description_snippet    457505
channel_id             461982
channel_title          461982
published_time         456559
status                 461982
dtype: int64

In [17]:
df_videos_channel = df_videos.groupby(["channel_title", "channel_id"]).size().sort_values(ascending=False).to_frame("total_count").reset_index()

In [18]:
df_processed_channel.head()

,channel_title,channel_id,unique_count
0,TEDx Talks,UCsT0YIqwnpJCM-mx7-gSA4Q,1289
1,Forbes Breaking News,UCg40OxZ1GYh3u3jBntB6DLg,762
2,VICE News,UCZaT_X_mc0BI-djXOlfhqWQ,684
3,Fox News,UCXIJgqnII2ZOINSWNOGFThA,673
4,PBS NewsHour,UC6ZFN9Tx6xh-skXCuRHCDpQ,403


In [19]:
df_videos_channel=df_videos_channel.merge(df_processed_channel[["channel_id", "unique_count"]], on="channel_id")

In [20]:
df_videos_channel.head()

,channel_title,channel_id,total_count,unique_count
0,Fox News,UCXIJgqnII2ZOINSWNOGFThA,42836,673
1,MIT OpenCourseWare,UCEBb1b_L6zDS3xTUrIALZOw,19836,277
2,The B1M,UC6n8I1UDTKP1IWjQMg6_TwA,16270,117
3,Dr. John Campbell,UCF9IOB2TExg3QIBupFtBDxg,14919,156
4,Forbes Breaking News,UCg40OxZ1GYh3u3jBntB6DLg,14445,762


In [6]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "client_secret_263597259024-fi9beqgg6a1pk5p457gjg86s53gf0ikh.apps.googleusercontent.com.json"

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=263597259024-fi9beqgg6a1pk5p457gjg86s53gf0ikh.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=QtlnO67wrxVmw8OpotaCTHC82w803T&prompt=consent&access_type=offline


Enter the authorization code:  4/1AX4XfWjVMVB0w0DvemH729kle2ZzQ50nz52i81gYcxut4oSgh_j-JPglwI8


In [18]:
def getUploads(channel):
    #print("Requested:", channel)
    request = youtube.channels().list(
        part="contentDetails",
        id=channel,
        maxResults=5
    )
    response = request.execute()

    if response and "items" in response:
        uploads = response.get("items")[0].get("contentDetails").get("relatedPlaylists").get("uploads")
        #print("Uploads:", uploads)
        return(uploads)
    else:
        #print("Returning Not found")
        return("Not fonud")

In [25]:
def getVideos(uploads, filename):
    #print(channel)
    playlist = Playlist(uploads)

    if playlist.hasMoreVideos:
        playlist.getNextVideos()
    
    for row in playlist.videos:
        video = row["id"]
        title = row["title"]
        duration = row["duration"]
        view_count = 0
        width = 0
        height = 0
        desciption_snippet = row["title"]
        channel_id = row["channel"]["id"]
        channel_title = row["channel"]["name"]
        published_time = "9999 days ago"
        status = 'Success'

        with open(file_name, mode='a') as out_file:
            result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow([video, title, duration, view_count, width, height, desciption_snippet, channel_id, channel_title, published_time, status])

In [26]:
df_videos_channel = df_videos_channel[df_videos_channel["total_count"]>=400]

In [27]:
df_videos_channel["uploads"] = df_videos_channel["channel_id"].apply(getUploads)

In [31]:
print(len(df_videos_channel))
df_videos_channel.head()

151


,channel_title,channel_id,total_count,unique_count,uploads
0,Fox News,UCXIJgqnII2ZOINSWNOGFThA,42836,673,UUXIJgqnII2ZOINSWNOGFThA
1,MIT OpenCourseWare,UCEBb1b_L6zDS3xTUrIALZOw,19836,277,UUEBb1b_L6zDS3xTUrIALZOw
2,The B1M,UC6n8I1UDTKP1IWjQMg6_TwA,16270,117,UU6n8I1UDTKP1IWjQMg6_TwA
3,Dr. John Campbell,UCF9IOB2TExg3QIBupFtBDxg,14919,156,UUF9IOB2TExg3QIBupFtBDxg
4,Forbes Breaking News,UCg40OxZ1GYh3u3jBntB6DLg,14445,762,UUg40OxZ1GYh3u3jBntB6DLg


In [29]:
date = time.strftime("%Y-%m-%d_%H_%M_%S")
file_name = "processed/unique_channels_uploads" + date + ".csv"

df_videos_channel.to_csv(file_name, index=False)

In [30]:
date = time.strftime("%Y-%m-%d_%H:%M:%S")
file_name = "processed/training_videos" + date + ".csv"

with open(file_name, mode='w') as out_file:
    result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    result_writer.writerow(['video', 'title', 'duration', 'view_count', 'width', 'height', 'description_snippet', 'channel_id', 'channel_title', 'published_time', 'status'])

In [32]:
i = 0
for index, row in df_videos_channel.iterrows():
    getVideos(row["uploads"], file_name)
    if i % 50 == 0: 
        print(i)
    i += 1

0
50
100
150


In [9]:
# Get channels
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("unique_channels.csv")]
df_videos_channel = pd.concat(map(pd.read_csv, filepaths))

In [21]:
def getVideosScaled(uploads, scale, filename):
    #print(channel)
    playlist = Playlist(uploads)
 
    i = 1
    
    while i <= scale:
        i += 1
        if playlist.hasMoreVideos:
            playlist.getNextVideos()
    
    for row in playlist.videos:
        video = row["id"]
        title = row["title"]
        duration = row["duration"]
        view_count = 0
        width = 0
        height = 0
        desciption_snippet = row["title"]
        channel_id = row["channel"]["id"]
        channel_title = row["channel"]["name"]
        published_time = "9999 days ago"
        status = 'Success'

        with open(file_name, mode='a') as out_file:
            result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow([video, title, duration, view_count, width, height, desciption_snippet, channel_id, channel_title, published_time, status])

In [10]:
df_videos_channel.head()

,channel_title,channel_id,total_count,unique_count,bias
0,Fox News,UCXIJgqnII2ZOINSWNOGFThA,42836,673,R
1,MIT OpenCourseWare,UCEBb1b_L6zDS3xTUrIALZOw,19836,277,C
2,Forbes Breaking News,UCg40OxZ1GYh3u3jBntB6DLg,14445,762,RC
3,TEDx Talks,UCsT0YIqwnpJCM-mx7-gSA4Q,11599,1288,C
4,Vanity Fair,UCIsbLox_y9dCIMLd8tdC6qg,9100,69,L


In [13]:
df_videos_channel = df_videos_channel[df_videos_channel["total_count"]>=400]
df_videos_channel.bias.value_counts()

LC    32
C     12
RC     6
L      5
R      1
Name: bias, dtype: int64

In [19]:
df_videos_channel["uploads"] = df_videos_channel["channel_id"].apply(getUploads)

In [15]:
date = time.strftime("%Y-%m-%d_%H_%M_%S")
file_name = "processed/training_videos" + date + ".csv"

with open(file_name, mode='w') as out_file:
    result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    result_writer.writerow(['video', 'title', 'duration', 'view_count', 'width', 'height', 'description_snippet', 'channel_id', 'channel_title', 'published_time', 'status'])

In [22]:
i = 0
for index, row in df_videos_channel.iterrows():
    bias = row["bias"]
    if bias == "LC":
        scale = 1
    elif bias == "C":
        scale = 3
    elif bias == "RC":
        scale = 5
    elif bias == "L":
        scale = 6
    else:
        scale = 30
        
    getVideosScaled(row["uploads"], scale, file_name)
    if i % 50 == 0: 
        print(i)
    i += 1

0
50


In [ ]:
round()